# Installing requirements

In [1]:
from tabulate import tabulate
import mysql.connector as mysql
from datetime import datetime
import os
import pandas as pd

In [2]:
os.chdir("..")

In [3]:
os.getcwd()

'c:\\Users\\sarac\\OneDrive - Università degli Studi di Milano\\[erasmus]\\[COURSES]\\[DB]\\assignments\\assignment2\\GITHUB\\Very_large_distributed_data_volumes_exercise2'

In [4]:
#path = os.getcwd()
#os.chdir(path+"/code")

In [5]:
#os.getcwd()

In [6]:
#%pip install -r requirements.txt

In [7]:
#os.chdir("..")
#os.getcwd()

# Class DBConnector

In [8]:
class DbConnector:
    """
    Connects to the MySQL server on the Ubuntu virtual machine.
    Connector needs HOST, DATABASE, USER and PASSWORD to connect,
    while PORT is optional and should be 3306.

    Example:
    HOST = "tdt4225-00.idi.ntnu.no" // Your server IP address/domain name
    DATABASE = "testdb" // Database name, if you just want to connect to MySQL server, leave it empty
    USER = "testuser" // This is the user you created and added privileges for
    PASSWORD = "test123" // The password you set for said user
    """

    def __init__(self,
                 HOST="tdt4225-19.idi.ntnu.no",
                 DATABASE="db_19",
                 USER="team19",
                 PASSWORD="team19*"):
        # Connect to the database
        try:
            self.db_connection = mysql.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD, port=3306)
        except Exception as e:
            print("ERROR: Failed to connect to db:", e)

        # Get the db cursor
        self.cursor = self.db_connection.cursor()

        print("Connected to:", self.db_connection.get_server_info())
        # get database information
        self.cursor.execute("select database();")
        database_name = self.cursor.fetchone()
        print("You are connected to the database:", database_name)
        print("-----------------------------------------------\n")

    def close_connection(self):
        # close the cursor
        self.cursor.close()
        # close the DB connection
        self.db_connection.close()
        print("\n-----------------------------------------------")
        print("Connection to %s is closed" % self.db_connection.get_server_info())


# Task 1

In [9]:
# we go to the Data folder
path = os.getcwd()
os.chdir(path+"/dataset/dataset/Data")
print(os.getcwd())

c:\Users\sarac\OneDrive - Università degli Studi di Milano\[erasmus]\[COURSES]\[DB]\assignments\assignment2\GITHUB\Very_large_distributed_data_volumes_exercise2\dataset\dataset\Data


In [10]:
def get_info_from_labels_txt_file() :
    user_id_list = []
    transportation_mode_list = []
    start_date_time_list = []
    end_date_time_list = []

    for dirpath, dirnames, filenames in os.walk(os.getcwd()):
        for filename in filenames:
            # if there is a labels.txt file, we save the info
            if filename.endswith('.txt'):
                try :
                    with open(os.path.join(dirpath, filename)) as f:
                        lines = f.readlines()

                        # skip the header
                        lines = lines[1:]

                        for line in lines:
                            
                            # we get the info of one line
                            data = line.split()

                            # we save each information into the correct list
                            last_directory_name = os.path.basename(dirpath)
                            user_id_list.append(last_directory_name)

                            transportation_mode_list.append(data[4])

                            start_date = data[0]
                            start_time = data[1]
                            end_date = data[2]
                            end_time = data[3]
                            start_datetime_str = start_date + " " + start_time
                            end_datetime_str = end_date + " " + end_time
                            start_datetime_str = start_datetime_str.replace('/', '-')
                            end_datetime_str = end_datetime_str.replace('/', '-')
                            combined_start_datetime = datetime.strptime(start_datetime_str, "%Y-%m-%d %H:%M:%S")
                            combined_end_datetime = datetime.strptime(end_datetime_str, "%Y-%m-%d %H:%M:%S")

                            start_date_time_list.append(combined_start_datetime)
                            end_date_time_list.append(combined_end_datetime)
                    f.close()

                # error handling
                except FileNotFoundError:
                    print(f"the file {filename} doesn't exist.")
                except Exception as e:
                    print(f"Error: {e}")

    return user_id_list, transportation_mode_list, start_date_time_list, end_date_time_list

In [11]:
def get_info_from_plt_file() :
    activity = 0
    activity_id_list = []
    user_id_list = []
    lat_list = []
    long_list = []
    altitude_list = []
    date_days_list = []
    current_date_time_list = []
    start_date_time_list = []
    end_date_time_list = []

    for dirpath, dirnames, filenames in os.walk(os.getcwd()):
        for filename in filenames:

            # we get the information of each plt file
            if filename.endswith('.plt'):
                try :
                    with open(os.path.join(dirpath, filename)) as f:
                        lines = f.readlines()

                        # skip the first 6 lines
                        lines = lines[6:]

                        # check the length of the plt file
                        if len(lines) <= 2500:

                            # we get the start and end date of each plt file
                            start_line= lines[0].split(',')
                            start_date = start_line[5]
                            start_time = start_line[6]
                            start_datetime = start_date + ' ' + start_time
                            start_datetime = start_datetime.rstrip('\n')
                            start_datetime = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")

                            end_line = lines[len(lines)-1].split(',')
                            end_date = end_line[5]
                            end_time = end_line[6]
                            end_datetime = end_date + ' ' + end_time
                            end_datetime = end_datetime.rstrip('\n')
                            end_datetime = datetime.strptime(end_datetime, "%Y-%m-%d %H:%M:%S")
    
                            # we save the information of each line of the plt file
                            for line in lines:

                                data = line.split(',')

                                activity_id_list.append(activity)

                                parent_directory = os.path.dirname(dirpath)
                                directory_name = os.path.basename(parent_directory) 
                                user_id_list.append(directory_name)

                                lat_list.append(float(data[0]))
                                long_list.append(float(data[1]))
                                altitude_list.append(int(float(data[3])))
                                date_days_list.append(float(data[4]))

                                date = data[5]
                                time = data[6]
                                datetime_draft = date + ' ' + time
                                datetime_draft = datetime_draft.rstrip('\n')
                                combined_datetime = datetime.strptime(datetime_draft, "%Y-%m-%d %H:%M:%S")
                                current_date_time_list.append(combined_datetime)

                                start_date_time_list.append(start_datetime)
                                end_date_time_list.append(end_datetime)

                    f.close()
                    activity +=1
                  
                except FileNotFoundError:
                    print(f"the file {filename} doesn't exist.")
                except Exception as e:
                    print(f"Error: {e}")
                    
    return activity_id_list, user_id_list, lat_list, long_list, altitude_list, date_days_list, current_date_time_list, start_date_time_list, end_date_time_list

In [12]:
def creating_activity_and_trackpoint_dataframe():    
    # getting info from txt files
    user_id_list, transportation_mode_list, start_date_time_list, end_date_time_list = get_info_from_labels_txt_file()
    labels_txt = {'user_id': user_id_list, 'transportation_mode': transportation_mode_list, 'start_datetime': start_date_time_list, 'end_datetime': end_date_time_list}
    labels_txt_df = pd.DataFrame(labels_txt)

    # getting info from plt files
    activity_id_list, user_id_list_2, lat_list, long_list, altitude_list, date_days_list, current_date_time_list, start_date_time_list2, end_date_time_list2 = get_info_from_plt_file()
    plt = {'activity_id': activity_id_list, 'user_id': user_id_list_2, 'lat': lat_list, 'long': long_list, 'altitude': altitude_list, 'date_days': date_days_list, 'current_date_time': current_date_time_list, 'start_datetime': start_date_time_list2, 'end_datetime': end_date_time_list2}
    plt_df = pd.DataFrame(plt)

    # merging both dataframes
    merged_df = pd.merge(plt_df, labels_txt_df, on=['user_id', 'start_datetime', 'end_datetime'], how='left')

    # creating activity table
    activity_table = merged_df[['activity_id','user_id', 'transportation_mode', 'start_datetime', 'end_datetime']]
    activity_table = activity_table.fillna("missing")
    activity_table['start_datetime'] = activity_table['start_datetime'].astype(str) # converting into string to be able to insert into the sql table
    activity_table['end_datetime'] = activity_table['end_datetime'].astype(str)
    activity_table = activity_table.drop_duplicates()

    # creating trackpoint table
    trackpoint_table = merged_df[['activity_id','lat', 'long', 'altitude', 'date_days', 'current_date_time']]
    trackpoint_table.rename(columns={'current_date_time': 'date_time'}, inplace=True)
    trackpoint_table['id'] = range(1, len(trackpoint_table) + 1)
    trackpoint_table = trackpoint_table[['id'] + [col for col in trackpoint_table.columns if col != 'id']]
    trackpoint_table['date_time'] = trackpoint_table['date_time'].astype(str)

    return activity_table, trackpoint_table


In [13]:
class Task1:

    # we use constant for defining the tables
    USER = 'User'
    ACTIVITY = 'Activity'
    TRACK_POINT = 'TrackPoint'

    def __init__(self):
        self.connection = DbConnector()
        self.db_connection = self.connection.db_connection
        self.cursor = self.connection.cursor

# --------------- CREATION OF THE TABLES ---------------

    def create_table_user(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                    userID VARCHAR(30) NOT NULL PRIMARY KEY,
                    has_labels BOOLEAN)
                """
        # this adds table_name to the %s variable and executes the query
        self.cursor.execute(query % self.USER)
        self.db_connection.commit()
    

    def create_table_activity(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                    activityID int NOT NULL,
                    userID varchar(30) NOT NULL,	
                    transportation_mode varchar(30),
                    start_date_time datetime NOT NULL,
                    end_date_time datetime NOT NULL,
                    PRIMARY KEY (activityID, transportation_mode)
                
                )"""
        # FOREIGN KEY (userID) REFERENCES User(userID)
        # this adds table_name to the %s variable and executes the query
        self.cursor.execute(query % self.ACTIVITY)
        self.db_connection.commit()

        query_constr = """ ALTER TABLE %s
                    ADD CONSTRAINT check_start_end_dates
                    CHECK (start_date_time < end_date_time)
                """

        self.cursor.execute(query_constr % self.ACTIVITY)
        self.db_connection.commit()
    

    def create_table_trackpoint(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                    trackPointID INT NOT NULL,
                    activityID int NOT NULL,
	                lat double NOT NULL,
	                lon double NOT NULL,
	                altitude int NOT NULL,
	                date_days double NOT NULL,
	                date_time datetime NOT NULL,
	                PRIMARY KEY (trackPointID))
                """
        # FOREIGN KEY (activityID) REFERENCES Activity(activityID))
        # this adds table_name to the %s variable and executes the query
        self.cursor.execute(query % self.TRACK_POINT)
        self.db_connection.commit()

# --------------- INSERT OF DATA ---------------

# *** USER ***
    def insert_data_user(self):
        print('*** INSERT IN USER ***')
        # we go to the Data directory
        os.chdir('/Users/victoriastasik/Documents/Very_large_distributed_data_volumes_exercise2/dataset/dataset/Data') #FIXME: maybe you need to change if

        # we get the list of all the diferent directory names (users ids) and we sort the list
        usersID =  os.listdir()
        usersID.sort()

        # we go back to the dataset directory and we read the labeled_ids.txt file
        os.chdir("..")
        with open('labeled_ids.txt') as f:
            labeled_ids = f.readlines()
        f.close()
        os.chdir('/Users/victoriastasik/Documents/Very_large_distributed_data_volumes_exercise2/dataset/dataset/Data')
        
        # # we delete the \n in each string
        for i in range (0, len(labeled_ids)):
            labeled_ids[i] = labeled_ids[i].strip()

        # we check if each user has a label or not and we save the info in a list
        # the indexes of has_labels and id lists are correponding
        has_labels = []
        for i in usersID : 
            if i in labeled_ids : 
                has_labels.append(True)
            else:
                has_labels.append(False)

        user_table = {'userID': usersID, 'has_labels': has_labels}
        user_dataframe = pd.DataFrame(user_table)
        user_data = user_dataframe.to_records(index=False).tolist()
        query = "INSERT INTO {} (userID, has_labels) VALUES (%s, %s)".format(self.USER)
        self.cursor.executemany(query, user_data)
        self.db_connection.commit()

    
# *** ACTIVITY ***
    def insert_data_activity(self, activity_dataframe):
        print('*** INSERT IN ACTIVITY ***')
        activity_data = activity_dataframe.to_records(index=False).tolist()
        query = "INSERT INTO {} (activityID, userID, transportation_mode, start_date_time, end_date_time) VALUES (%s, %s, %s, %s, %s)".format(self.ACTIVITY)

        # Batch insert (e.g., insert 100 rows at a time)
        batch_size = 1000
        for i in range(0, len(activity_data), batch_size):
            batch_data = activity_data[i : i+batch_size]
            self.cursor.executemany(query, batch_data)
            self.db_connection.commit()


# *** TRACKPOINT ***
    def insert_data_trackpoint(self, trackpoint_dataframe):
        print('*** INSERT IN TRACKPOINT ***')
        trackpoint_data = trackpoint_dataframe.to_records(index=False).tolist()
        query = "INSERT INTO {} (trackpointID, activityID, lat, lon, altitude, date_days, date_time) VALUES (%s, %s, %s, %s, %s, %s, %s)".format(self.TRACK_POINT)

        # Batch insert (e.g., insert 100 rows at a time)
        batch_size = 1000
        for i in range(0, len(trackpoint_data), batch_size):
            batch_data = trackpoint_data[i : i+batch_size]
            self.cursor.executemany(query, batch_data)
            self.db_connection.commit()  
       
            

# --------------- FETCH DATA ---------------

    def execute_query(self, query, query_name):
        self.cursor.execute(query)
        rows = self.cursor.fetchall()
        # Using tabulate to show the table in a nice way
        print(query_name)
        print(tabulate(rows, headers=self.cursor.column_names))
        print('\n')
        print('******************************************************')
        return rows
    

# ---------------- ALTER TABLES ----------------

    def altering_tables_with_foreign_key(self):
        
        query1 = """ALTER TABLE Activity
                ADD CONSTRAINT fk_activity
                FOREIGN KEY (userID)
                REFERENCES User(userID);"""
        self.cursor.execute(query1) 
        self.db_connection.commit() 

        query2 = """ALTER TABLE TrackPoint
                ADD CONSTRAINT fk_trackpoint
                FOREIGN KEY (activityID)
                REFERENCES Activity(activityID);"""
        self.cursor.execute(query2) 
        self.db_connection.commit()

# --------------- DROP TABLE ---------------

    def drop_table(self, table_name):
        print("Dropping table %s..." % table_name)
        query = "DROP TABLE %s"
        self.cursor.execute(query % table_name)

# --------------- SHOW TABLES ---------------

    def show_tables(self):
        self.cursor.execute("SHOW TABLES")
        rows = self.cursor.fetchall()
        print(tabulate(rows, headers=self.cursor.column_names))


# Task 2 - *queries*

**GENERAL QUERIES**


In [14]:
select_table_activity = """
        SELECT * FROM Activity LIMIT 10
        """

In [15]:
select_table_user = """
        SELECT * FROM User LIMIT 11
"""

In [16]:
select_table_trackpoint = """
        SELECT * FROM TrackPoint LIMIT 10
"""

**QUERY 1**
How many users, activities and trackpoints are there in the dataset (after it is inserted into the database).

In [17]:
query1 = """
        SELECT
            (SELECT COUNT(*) FROM User) AS TotUser,
            (SELECT COUNT(*) FROM Activity) AS TotActivity,
            (SELECT COUNT(*) FROM TrackPoint) AS TotTrackPoint;
        """


**QUERY 2** Find the average, maximum and minimum number of trackpoints per user.

In [18]:
query2 =  """
        SELECT 
            userID, 
            avg(nb_tp_per_activity) as mean_trackpoint, 
            max(nb_tp_per_activity) as max_trackpoint, 
            min(nb_tp_per_activity) as min_trackpoint
        FROM (select count(*) as nb_tp_per_activity, activityID
        FROM TrackPoint as tp
        GROUP BY activityID) tp
        LEFT JOIN Activity a 
        ON tp.activityID = a.activityID
        GROUP BY userID
        LIMIT 10
    """


**QUERY 3** Find the top 15 users with the highest number of activities.

In [19]:
query3 = """
        SELECT U.userID, count(A.activityID) AS activity_nb 
        FROM User U LEFT JOIN Activity A ON U.userID = A.userID 
        GROUP BY U.userID 
        ORDER BY activity_nb DESC 
        """

**QUERY 4** Find all users who have taken a bus.

In [20]:
query4 =  """
        SELECT DISTINCT userID
        FROM Activity
        WHERE transportation_mode = 'bus' 
        ORDER BY LOWER(userID)
        """

**QUERY 5** List the top 10 users by their amount of different transportation modes.

In [21]:
query5 = """
    SELECT userID,
    COUNT(transportation_mode) as nb_transportation_mode
    FROM(
        SELECT 
            userID,
            transportation_mode
        FROM Activity
        GROUP BY UserID,transportation_mode) t
    GROUP BY UserID
    ORDER BY nb_transportation_mode DESC
"""

**QUERY 6** Find activities that are registered multiple times. You should find the query even if it gives zero result.


In [22]:
query6 = """
    SELECT 
        A.userID, 
        A.transportation_mode, 
        A.start_date_time, 
        A.end_date_time, 
        COUNT(*) AS registration_nb 
    FROM Activity A
    GROUP BY 
        A.userID, 
        A.transportation_mode, 
        A.start_date_time, 
        A.end_date_time HAVING COUNT(*) > 1;
"""

**QUERY 7** 
1. Find the number of users that have started an activity in one day and ended the activity the next day.
2. List the transportation mode, user id and duration for these activities.

In [23]:
#info: DATE extracts the "date" part
query7_a = """
        SELECT COUNT(DISTINCT userID) as tot_users_with_activity
        FROM Activity
        WHERE 
                DATE(start_date_time) != DATE(end_date_time) 
                and 
                DATEDIFF(end_date_time, start_date_time) = 1
        """

In [24]:
query7_b = """
    SELECT 
        userID, 
        transportation_mode, 
        TIMESTAMPDIFF(MINUTE, start_date_time, end_date_time) as duration
    FROM Activity
    WHERE 
        DATE(start_date_time) != DATE(end_date_time) 
        and 
        DATEDIFF(end_date_time, start_date_time) = 1
    ORDER BY userID ASC, duration DESC
"""

**QUERY 8** Find the number of users which have been close to each other in time and space.
Close is defined as the same space (50 meters) and for the same half minute (30 seconds)


In [25]:
query8 = """
    SELECT 
        user1,
        user2, 
        COUNT (
            CASE WHEN (ST_Distance(point(long1, lat1),
 			point(long2, lat2)) <= 50) 
            and (abs(TIMESTAMPDIFF(Second,date1,date2))<30) 
            THEN 1 ELSE null end) as nb
    FROM (
        SELECT 
            tp1.userID as user1, 
            tp1.lat as lat1,
            tp1.lon as long1,
            tp1.date_time as date1,
            tp2.userID as user2, 
            tp2.lat as lat2,
            tp2.lon as long2,
            tp2.date_time as date2
        FROM (
            SELECT a.userID,tp.lat, tp.lon, tp.date_time 
            FROM TrackPoint tp 
            JOIN Activity a 
            ON a.activityID=tp.activityID)tp1 
            JOIN (
                SELECT a.userID, tp.lat, tp.lon,tp.date_time 
                FROM TrackPoint tp 
                JOIN Activity a 
                ON a.activityID=tp.activityID) tp2
                ON date(tp1.date_time) = date(tp1.date_time)
                WHERE tp1.userID<tp2.userID) tp_compare
    GROUP BY user1,user2;

"""

**QUERY 9** Find the top 15 users who have gained the most altitude meters.

In [26]:
query9 = """
    SET @prev_userID = NULL;
    SET @prev_altitude = NULL;
    
    SELECT userID,  SUM(gained_altitude) AS total_altitude 
    FROM (
            SELECT userID, altitude, 
            CASE 
                WHEN @prev_userID IS NULL OR userID <> @prev_userID 
                THEN 0 WHEN altitude > @prev_altitude 
                THEN altitude - @prev_altitude 
                ELSE 0 END AS gained_altitude, @prev_userID := userID, @prev_altitude := altitude 
            FROM TrackPoint JOIN Activity ON TrackPoint.activityID = Activity.activityID 
            ORDER BY userID, trackPointID) AS Tab 
    GROUP BY userID ORDER BY total_altitude DESC 
    LIMIT 15; 
"""

**QUERY 10** Find the users that have traveled the longest total distance in one day for each transportation mode.


In [33]:
query10 = """
SELECT t.userID, t.transportation_mode, t.dis
    FROM (
        SELECT 
            userID, 
            transportation_mode, 
            dis,
            ROW_NUMBER() OVER (PARTITION BY transportation_mode ORDER BY dis DESC) AS RowNum
        FROM (
            SELECT
                userID,
                transportation_mode,
                max(dis) as dis
            FROM (
                SELECT 
                    userID,
                    date,
                    transportation_mode, 
                    sum(distance) as dis
                FROM (
                    SELECT 
                        tp_max.*,
                        tp2.lon as lon_min,
                        tp2.lat as lat_min,
                        ST_Distance(point(lon_max, lat_max),point(tp2.lon, tp2.lat)) as distance 
                    FROM (
                        SELECT 
                            ac_min_max.*,
                            tp1.lon as lon_max,
                            tp1.lat as lat_max
                        FROM (
                            SELECT 
                                max(trackpointID) as max,
                                min(trackpointID) as min,
                                activityID
                            FROM (
                                SELECT *
                                FROM TrackPoint
                                ORDER BY trackpointID)tp
                            GROUP BY activityID)ac_min_max
                        LEFT JOIN TrackPoint tp1
                        ON ac_min_max.max=tp1.trackpointID)tp_max
                    LEFT JOIN TrackPoint tp2
                    ON tp_max.min=tp2.trackpointID)tab
                RIGHT JOIN (
                    SELECT 
                        userID,
                        activityID,
                        transportation_mode,
                        date(start_date_time) as date 
                    FROM Activity 
                    WHERE date(start_date_time)=date(end_date_time) ) a 
                ON a.activityID= tab.activityID
                GROUP BY userID,date,transportation_mode)tt
            GROUP BY transportation_mode,userID
            ORDER BY max(dis) DESC
            )tot
    )t
WHERE t.RowNum = 1;
"""

**QUERY 11** Find all users who have invalid activities, and the number of invalid activities per user.


In [30]:
query11 = """
                SET @prev_activity = NULL;
                SET @prev_time = NULL;
               
                SELECT 
                    userID, 
                    activityID,
                    trackpointID, 
                    date_time, 
                    CASE 
                        WHEN @prev_activity is null or @prev_activity<>activityID THEN 0
                        WHEN TIMESTAMPDIFF(MINUTE,date_time,@prev_time)>5 THEN 1
                        ELSE 0 end AS flag_incorect,
                    @prev_activity = activityID,
                    @prev_time = date_time
                FROM TrackPoint tp
                JOIN Activity a ON tp.activityID = a.activityID
                ORDER BY activityID,trackpointID

"""

**QUERY 12** Find all users who have registered transportation_mode and their most used transportation_mode.

In [31]:
query12 = """
        SET @prev_activity = NULL;
        SET @prev_time = NULL;
                    
        SELECT 
            userID, 
            activityID,
            trackpointID, 
            date_time, 
            CASE 
                WHEN @prev_activity is null or @prev_activity<>activityID THEN 0
                WHEN TIMESTAMPDIFF(MINUTE,date_time,@prev_time)>5 THEN 1
                ELSE 0 end AS flag_incorect,
            @prev_activity = activityID,
            @prev_time = date_time
        FROM TrackPoint tp
        JOIN Activity a ON tp.activityID = a.activityID
        ORDER BY activityID,trackpointID
"""

# Main

In [34]:
def main():
    
    program = None

    try:
        program = Task1() 
        #activity_table, trackpoint_table = creating_activity_and_trackpoint_dataframe()

        # --- DROP TABLE ---
        # program.drop_table(program.TRACK_POINT)
        # program.drop_table(program.ACTIVITY)
        # program.drop_table(program.USER)

        # --- TABLE CREATION ---
        # program.create_table_user()
        # program.create_table_activity()
        # program.create_table_trackpoint()
        
        
        # --- INSERTION ---
        # program.insert_data_user()
        # program.insert_data_activity(activity_table)
        # program.insert_data_trackpoint(trackpoint_table)

        # --- ALTER TABLES ---
        # program.altering_tables_with_foreign_key()
            
        # --- SHOW TABLES ---
        # program.show_tables()

        # --- EXECUTE QUERIES ---
       
        # program.execute_query(select_table_user, '\nSELECT USERS')
        # program.execute_query(select_table_activity, '\nSELECT ACTIVITY')
        # program.execute_query(select_table_trackpoint, '\nSELECT TRACKPOINT')
        # program.execute_query(query1, '\nQUERY 1')
        # program.execute_query(query2, '\nQUERY 2')
        # program.execute_query(query3, '\nQUERY 3')
        # program.execute_query(query4, '\nQUERY 4')
        # program.execute_query(query5, '\nQUERY 5')
        # program.execute_query(query6, '\nQUERY 6')
        # program.execute_query(query7_a, '\nQUERY 7 A')
        # program.execute_query(query7_b, '\nQUERY 7 B')
        # program.execute_query(query8, '\nQUERY 8')
        #program.execute_query(query9, '\nQUERY 9')
        program.execute_query(query10, '\nQUERY 10')
        # program.execute_query(query10, '\nQUERY 10')
        # program.execute_query(query11, '\nQUERY 11')
        # program.execute_query(query12, '\nQUERY 12')
        
    except Exception as e:
        print("ERROR: Failed to use database:", e)
    finally:
        if program:
            program.connection.close_connection()

if __name__ == '__main__':
    main()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('db_19',)
-----------------------------------------------


QUERY 10
  userID  transportation_mode             dis
--------  ---------------------  ------------
     128  airplane                10.2009
     128  bike                     0.313325
     128  boat                     0.586744
     128  bus                      1.45441
     128  car                      3.45451
     128  missing                248.996
     062  run                      0.00020134
     128  subway                   0.245041
     128  taxi                     0.234522
     062  train                    2.99243
     108  walk                     0.25725


******************************************************

-----------------------------------------------
Connection to None is closed
